# Filter data fields in a VCF

## Example VCF file

We need an example VCF file for demonstation. You can manually download it from [link](http://faculty.washington.edu/browning/beagle/test.08Jun17.d8b.vcf.gz) (877KB) and put the file in your current working directory. Or, within Julia, 

In [1]:
isfile("test.08Jun17.d8b.vcf.gz") || download("http://faculty.washington.edu/browning/beagle/tcest.08Jun17.d8b.vcf.gz", 
    joinpath(pwd(), "test.08Jun17.d8b.vcf.gz"))
stat("test.08Jun17.d8b.vcf.gz")

StatStruct(mode=0o100644, size=876514)

The first 35 lines of the VCF file are

In [2]:
using VCFTools

fh = openvcf("test.08Jun17.d8b.vcf.gz", "r")
for l in 1:35
    println(readline(fh))
end
close(fh)

##fileformat=VCFv4.1
##INFO=<ID=LDAF,Number=1,Type=Float,Description="MLE Allele Frequency Accounting for LD">
##INFO=<ID=AVGPOST,Number=1,Type=Float,Description="Average posterior probability from MaCH/Thunder">
##INFO=<ID=RSQ,Number=1,Type=Float,Description="Genotype imputation quality from MaCH/Thunder">
##INFO=<ID=ERATE,Number=1,Type=Float,Description="Per-marker Mutation rate from MaCH/Thunder">
##INFO=<ID=THETA,Number=1,Type=Float,Description="Per-marker Transition rate from MaCH/Thunder">
##INFO=<ID=CIEND,Number=2,Type=Integer,Description="Confidence interval around END for imprecise variants">
##INFO=<ID=CIPOS,Number=2,Type=Integer,Description="Confidence interval around POS for imprecise variants">
##INFO=<ID=END,Number=1,Type=Integer,Description="End position of the variant described in this record">
##INFO=<ID=HOMLEN,Number=.,Type=Integer,Description="Length of base pair identical micro-homology at event breakpoints">
##INFO=<ID=HOMSEQ,Number=.,Type=String,Description="Seque

As in typical VCF files, it has a bunch of meta-information lines, one header line, and then one line for each each marker. At least for the first couple of markers, genetic data has fields GT (genotype), DS (dosage), and GL (genotype likelihood). In total there are 1356 markers and 191 individuals.

In [3]:
nrecords("test.08Jun17.d8b.vcf.gz"), nsamples("test.08Jun17.d8b.vcf.gz")

(1356, 191)

## Extract a single field

If we are only interested in the genotype (GT) data in this VCF file, we can call `filter_genotype` function for filtering.  
* The first argument is the source VCF.  
* The second argument is the output VCF.  
* The third argument is a vector of fields to output.

In [4]:
@time filter_genotype("test.08Jun17.d8b.vcf.gz", "test.gt.vcf.gz", ["GT"])

  2.043363 seconds (8.53 M allocations: 610.777 MiB, 9.66% gc time)


The first 35 lines of the resultant file are

In [5]:
fh = openvcf("test.gt.vcf.gz", "r")
for l in 1:35
    println(readline(fh))
end
close(fh)

##fileformat=VCFv4.1
##INFO=<ID=LDAF,Number=1,Type=Float,Description="MLE Allele Frequency Accounting for LD">
##INFO=<ID=AVGPOST,Number=1,Type=Float,Description="Average posterior probability from MaCH/Thunder">
##INFO=<ID=RSQ,Number=1,Type=Float,Description="Genotype imputation quality from MaCH/Thunder">
##INFO=<ID=ERATE,Number=1,Type=Float,Description="Per-marker Mutation rate from MaCH/Thunder">
##INFO=<ID=THETA,Number=1,Type=Float,Description="Per-marker Transition rate from MaCH/Thunder">
##INFO=<ID=CIEND,Number=2,Type=Integer,Description="Confidence interval around END for imprecise variants">
##INFO=<ID=CIPOS,Number=2,Type=Integer,Description="Confidence interval around POS for imprecise variants">
##INFO=<ID=END,Number=1,Type=Integer,Description="End position of the variant described in this record">
##INFO=<ID=HOMLEN,Number=.,Type=Integer,Description="Length of base pair identical micro-homology at event breakpoints">
##INFO=<ID=HOMSEQ,Number=.,Type=String,Description="Seque

## Extract more than one fields

To extract more than one fields, say genotypes (GT) and dosages (DS),

In [6]:
@time filter_genotype("test.08Jun17.d8b.vcf.gz", "test.gt.ds.vcf.gz", ["GT", "DS"])

  0.364364 seconds (4.28 M allocations: 414.152 MiB, 8.18% gc time)


The first 35 lines of the resultant file are

In [7]:
fh = openvcf("test.gt.ds.vcf.gz", "r")
for l in 1:35
    println(readline(fh))
end
close(fh)

##fileformat=VCFv4.1
##INFO=<ID=LDAF,Number=1,Type=Float,Description="MLE Allele Frequency Accounting for LD">
##INFO=<ID=AVGPOST,Number=1,Type=Float,Description="Average posterior probability from MaCH/Thunder">
##INFO=<ID=RSQ,Number=1,Type=Float,Description="Genotype imputation quality from MaCH/Thunder">
##INFO=<ID=ERATE,Number=1,Type=Float,Description="Per-marker Mutation rate from MaCH/Thunder">
##INFO=<ID=THETA,Number=1,Type=Float,Description="Per-marker Transition rate from MaCH/Thunder">
##INFO=<ID=CIEND,Number=2,Type=Integer,Description="Confidence interval around END for imprecise variants">
##INFO=<ID=CIPOS,Number=2,Type=Integer,Description="Confidence interval around POS for imprecise variants">
##INFO=<ID=END,Number=1,Type=Integer,Description="End position of the variant described in this record">
##INFO=<ID=HOMLEN,Number=.,Type=Integer,Description="Length of base pair identical micro-homology at event breakpoints">
##INFO=<ID=HOMSEQ,Number=.,Type=String,Description="Seque

Note the data fields in the output file is in alphabet order, which may be different from the order in original VCF.

## No found data fields

If the requested data fields are not found in the source VCF, no data is written to the ouptut VCF.

In [8]:
@time filter_genotype("test.08Jun17.d8b.vcf.gz", "test.dummy.vcf.gz", ["DUMMY"])
fh = openvcf("test.dummy.vcf.gz", "r")
for l in 1:35
    println(readline(fh))
end
close(fh)

  0.204777 seconds (2.70 M allocations: 288.384 MiB, 7.65% gc time)
##fileformat=VCFv4.1
##INFO=<ID=LDAF,Number=1,Type=Float,Description="MLE Allele Frequency Accounting for LD">
##INFO=<ID=AVGPOST,Number=1,Type=Float,Description="Average posterior probability from MaCH/Thunder">
##INFO=<ID=RSQ,Number=1,Type=Float,Description="Genotype imputation quality from MaCH/Thunder">
##INFO=<ID=ERATE,Number=1,Type=Float,Description="Per-marker Mutation rate from MaCH/Thunder">
##INFO=<ID=THETA,Number=1,Type=Float,Description="Per-marker Transition rate from MaCH/Thunder">
##INFO=<ID=CIEND,Number=2,Type=Integer,Description="Confidence interval around END for imprecise variants">
##INFO=<ID=CIPOS,Number=2,Type=Integer,Description="Confidence interval around POS for imprecise variants">
##INFO=<ID=END,Number=1,Type=Integer,Description="End position of the variant described in this record">
##INFO=<ID=HOMLEN,Number=.,Type=Integer,Description="Length of base pair identical micro-homology at event bre

## Subsetting VCF files

Sometimes we wish to subset entire VCF files, such as filtering out certain samples or records (SNPs). This is achieved via the `filter` function:

In [9]:
# filtering by specifying indices to keep
vcffile = "test.08Jun17.d8b.vcf.gz"              # a VCF file you want to filter
samples = nsamples(vcffile)
records = nrecords(vcffile)
record_mask = collect(1:records)                 # keep all records (SNPs)
sample_mask = collect(2:(samples - 1))           # keep all but first and last sample (individual)
destination = "filtered.test.08Jun17.d8b.vcf.gz" # output file name
@time VCFTools.filter(vcffile, record_mask, sample_mask, des=destination)

  0.747510 seconds (3.94 M allocations: 341.610 MiB, 3.33% gc time)


One can check that the resulting VCF file does not contain information for the first and last sample:

In [10]:
X = convert_gt(Float32, vcffile, as_minorallele=false)
X_filtered = convert_gt(Float32, destination, as_minorallele=false)
all(X[2:(samples - 1), :] .== X_filtered)

true

One can also supply bitvectors as masks:

In [11]:
record_mask = trues(records)
sample_mask = trues(samples)
record_mask[1] = record_mask[end] = false
@time VCFTools.filter(vcffile, record_mask, sample_mask, des=destination)

  0.536422 seconds (831.06 k allocations: 110.737 MiB, 3.02% gc time)


In [12]:
X = convert_gt(Float32, vcffile, as_minorallele=false)
X_filtered = convert_gt(Float32, destination, as_minorallele=false)
all(X[:, 2:(records - 1)] .== X_filtered)

true

## Masksing VCF files

Sometimes users may wish to mask certain entries of a VCF file for statistical analysis. The `mask_gt` function allows users to mask genotype data. A masked entry would become `./.` (default) in the output VCF file. 

In [13]:
using Random
vcffile = "test.08Jun17.d8b.vcf.gz"
samples = nsamples(vcffile)
records = nrecords(vcffile)
masks   = bitrand(records, samples) #masks[i] = true will convert the entry to missing

1356×191 BitArray{2}:
 0  0  0  0  1  1  1  1  0  0  1  1  0  …  0  0  1  0  0  0  0  0  1  1  0  0
 0  1  0  0  0  1  0  0  1  0  1  1  1     0  0  1  1  0  1  0  1  0  1  0  1
 1  0  0  0  0  1  0  1  1  1  0  1  0     1  0  1  0  1  1  0  0  1  1  1  1
 0  0  1  1  0  1  1  1  1  1  1  0  1     0  1  1  1  0  0  0  1  0  1  0  0
 0  1  1  1  1  0  0  1  1  0  1  1  0     0  0  0  1  1  1  0  1  0  0  1  1
 0  0  0  0  1  0  0  1  0  0  1  0  0  …  0  1  0  0  0  1  1  1  1  0  1  1
 0  1  1  1  1  1  1  0  1  1  1  0  1     1  1  0  1  1  0  0  0  1  0  0  1
 1  0  0  1  0  0  0  1  1  1  1  1  1     1  0  1  0  1  1  1  0  1  0  0  1
 0  0  1  0  1  1  1  0  0  0  0  1  0     0  1  1  1  1  0  0  0  1  1  0  0
 1  1  1  1  0  1  0  1  0  0  0  0  0     1  0  0  0  0  1  0  0  1  1  1  0
 1  1  1  1  1  0  0  1  1  0  1  1  0  …  0  1  0  0  0  0  0  1  1  0  0  0
 1  0  1  1  0  0  0  1  1  1  1  0  0     1  0  0  0  0  1  1  0  0  0  0  1
 1  0  1  1  1  0  1  0  1  1  0  1  0    

In [14]:
mask_gt("test.08Jun17.d8b.vcf.gz", masks, des = "masked.vcf.gz", separator = '/') # separator can also be '|'

The first 35 lines of the resultant files are

In [15]:
fh = openvcf("masked.vcf.gz", "r")
for l in 1:35
    println(readline(fh))
end
close(fh)

##fileformat=VCFv4.1
##INFO=<ID=LDAF,Number=1,Type=Float,Description="MLE Allele Frequency Accounting for LD">
##INFO=<ID=AVGPOST,Number=1,Type=Float,Description="Average posterior probability from MaCH/Thunder">
##INFO=<ID=RSQ,Number=1,Type=Float,Description="Genotype imputation quality from MaCH/Thunder">
##INFO=<ID=ERATE,Number=1,Type=Float,Description="Per-marker Mutation rate from MaCH/Thunder">
##INFO=<ID=THETA,Number=1,Type=Float,Description="Per-marker Transition rate from MaCH/Thunder">
##INFO=<ID=CIEND,Number=2,Type=Integer,Description="Confidence interval around END for imprecise variants">
##INFO=<ID=CIPOS,Number=2,Type=Integer,Description="Confidence interval around POS for imprecise variants">
##INFO=<ID=END,Number=1,Type=Integer,Description="End position of the variant described in this record">
##INFO=<ID=HOMLEN,Number=.,Type=Integer,Description="Length of base pair identical micro-homology at event breakpoints">
##INFO=<ID=HOMSEQ,Number=.,Type=String,Description="Seque

Masked entries would be `missing` in the resulting matrix:

In [16]:
A = convert_gt(Int, "masked.vcf.gz")

191×1356 Array{Union{Missing, Int64},2}:
 0         0          missing  0         …   missing   missing   missing
 0          missing  0         0            0          missing  0       
 0         0         0          missing     0         0         0       
 0         0         0          missing     0          missing  0       
  missing  0         0         0             missing   missing  0       
  missing   missing   missing   missing  …  0         0         0       
  missing  0         0          missing      missing   missing  0       
  missing  0          missing   missing      missing   missing  0       
 0          missing   missing   missing     0          missing   missing
 0         0          missing   missing      missing  0         0       
  missing   missing  0          missing  …   missing  1          missing
  missing   missing   missing  0            0          missing   missing
 0          missing  0          missing     1          missing  0       
 ⋮        